# Hito 2

## 1. Preliminares

A continuación se detallan las características del problema a evaluar:

### 1.1 Problema (Analizando los crímenes en la ciudad de Nueva York)

Se trabajará con datos públkicos del departamento de policía de NY. Debemos reportar los siguientes resultados:

1) Probabilidad que un individuo sea arrestado en uno de los cinco barrios, condicional al género y a la raza

2) Modelo predictivo que prediga efectivamente si un determinado procedimiento concluirá en un arresto o nonlocal

3) Modelo predictivo para determinar si el procedimiento concluirá en una acción violenta

Para los modelos predictivos, tenemos:

* Variables objetivo:

* Atributos:

### 1.2 Modelos a implementar

### 1.3 Métricas y estrategia de división de muestras

### 1.4 Procesamiento tentativo

* Primero que todo se detalla que se utilizarán los datos del 2009 como conjunto de entrenamiento, y los datos de 2010 como conjunto de validación

* Eliminaremos la variable "year", dado que posee un único valor. Esto implica que no es una variable informativa sino estática

* Dado que existe un gran número de variables, utilizaremos la función "create_suitable_dataframe" detallada en el archivo "preproc_nyc_sqf.py" para hacer del conjunto de datos más amigable. Esto implica transformar variables categóricas, seleccionar variables, definir la edad del individuo, entre otros

* Definición de una acción violenta: cuando pf_hands, pf_wall, pf_grnd, pf_drwep, pf_ptwep, pf_baton, pf_hcuff, pf_pepsp, o pf_other; haga referencia a "yes". Es decir, cuando al menos una de las variables anteriores que hacen referencia al uso efectivo de la fuerza por parte del oficial asumiremos que hubo agresividad por alguna de las partes

## 2. Aspectos Computacionales

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import re

from static import *
from apoyo_prueba_2.preproc_nyc_sqf import *

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10, 6)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gonzalooportus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gonzalooportus/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/gonzalooportus/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gonzalooportus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 3. Feature Engineering

In [2]:
# lectura de datos
df = pd.read_csv('apoyo_prueba_2/2009_1perc.csv').drop('Unnamed: 0', axis=1)

### 3.1 Limpiamos el conjunto de datos

In [3]:
# eliminamos la variable year
df = df.drop('year', axis=1)

In [4]:
# aplicamos función para estructurar de mejor forma el conjunto de datos
proc_df, suitable_categorical_attributes, suitable_numerical_attributes  = create_suitable_dataframe(df)

['pct', 'ser_num', 'datestop', 'timestop', 'perstop', 'compyear', 'comppct', 'repcmd', 'revcmd', 'dob', 'age', 'ht_feet', 'ht_inch', 'weight', 'addrpct', 'linecm', 'detailcm', 'perobs']


### 3.2 Creamos la variable objetiv que se relaciona a la existencia de violencia en el arresto

In [5]:
# en cualquier circunstancia que hubo uso de fuerza bruta, entonces consideramos que hubo violencia en el arresto

condition = (proc_df['pf_hands'] == 'Y')|(proc_df['pf_wall'] == 'Y')|(proc_df['pf_grnd'] == 'Y')|(proc_df['pf_drwep'] == 'Y')\
    |(proc_df['pf_ptwep'] == 'Y')|(proc_df['pf_baton'] == 'Y')|(proc_df['pf_hcuff'] == 'Y')|(proc_df['pf_pepsp'] == 'Y')\
        |(proc_df['pf_other'] == 'Y')

In [6]:
proc_df['was_violent'] = np.where(condition, 1, 0)

### 3.3 Identificamos la calidad de los datos iniciales post primer pre-procesamiento

In [9]:
df_calidad = calidad_datos(proc_df)

In [14]:
print(f"Nº atributos con nulos: {df_calidad[df_calidad['porc_nulos'] > 0].shape[0]}")

Nº atributos con nulos: 0


In [16]:
df_calidad['max'].max()

12.0

Identificamos además que los valores de las variables se encuentran en un intervalo de valores similar, ya que el valor máximo entre todas es igual a 12. Esto nos da un buen inicio para no tener que normalizar los datos.

Además, no tenemos que trabajar sobre datos nulos ya que no existen.

# 4. Análisis Exploratorio